> **_NOTE:_**  **This script is supposed to be executed at SageMaker Notebook!**

## prerequesites
- We have setup an **SageMaker Notebook**, the **S3 bucket** to store the bindle, and config their permission

## Step 1
Use git to clone this file to your SageMaker Notebook instance, and open this run.ipynb at your SageMaker Notebook

## Step 2
Prepare the model file for SageMaker. Run below code blocks in sequence.

In [ ]:
!mkdir handler
!mkdir handler/code
!mkdir handler/MAR-INF

In [ ]:
%%writefile handler/code/requirements.txt
transformers==4.44.1
sentencepiece==0.1.99

In [ ]:
%%writefile handler/MAR-INF/MANIFEST.json
{
  "runtime": "python",
  "model": {
    "modelName": "neuralsparse",
    "handler": "neural_sparse_handler.py",
    "modelVersion": "1.0",
    "configFile": "neural_sparse_config.yaml"
  },
  "archiverVersion": "0.9.0"
}

In [ ]:
%%writefile handler/neural_sparse_config.yaml
## configs about dynamic batch inference
batchSize: 16
maxBatchDelay: 5
responseTimeout: 300

In [ ]:
%%writefile handler/neural_sparse_handler.py
import os
import re
import itertools
import json
import torch
import transformers
from ts.torch_handler.base_handler import BaseHandler

model_id = os.environ.get(
    "MODEL_ID", "opensearch-project/opensearch-neural-sparse-encoding-doc-v2-distill"
)
max_bs = int(os.environ.get("MAX_BS", 32))
prune_ratio = float(os.environ.get("PRUNE_RATIO", 0.1))

version_match = re.search(r"v(\d+)", model_id)
use_l0 = False
if version_match:
    version = int(version_match.group(1))
    use_l0 = version >= 3


class SparseEncodingModelHandler(BaseHandler):
    class SparseModel(torch.nn.Module):
        @staticmethod
        def from_pretrained(path):
            return SparseEncodingModelHandler.SparseModel(path)

        def __init__(self, model_id):
            super().__init__()
            self.backbone = transformers.AutoModelForMaskedLM.from_pretrained(model_id)
            self.special_token_ids = []

        def set_special_token_ids(self, special_token_ids):
            self.special_token_ids = special_token_ids

        def forward(self, **kwargs):
            output = self.backbone(**kwargs)[0]
            values, _ = torch.max(
                output * kwargs.get("attention_mask").unsqueeze(-1), dim=1
            )
            values = torch.log1p(torch.relu(values))
            if use_l0:
                values = torch.log1p(values)
            values[:, self.special_token_ids] = 0
            max_values = values.max(dim=-1)[0].unsqueeze(1) * prune_ratio
            return values * (values > max_values)

    class SparsePostProcessor(object):
        def __init__(self, tokenizer):
            self.tokenizer = tokenizer
            self.id_to_token = ["" for i in range(tokenizer.vocab_size)]
            for token, _id in tokenizer.vocab.items():
                self.id_to_token[_id] = token

        def __call__(self, sparse_vector):
            sample_indices, token_indices = torch.nonzero(sparse_vector, as_tuple=True)
            non_zero_values = sparse_vector[(sample_indices, token_indices)].tolist()
            number_of_tokens_for_each_sample = (
                torch.bincount(sample_indices).cpu().tolist()
            )
            tokens = [self.id_to_token[_id] for _id in token_indices.tolist()]

            output = []
            end_idxs = list(
                itertools.accumulate([0] + number_of_tokens_for_each_sample)
            )
            for i in range(len(end_idxs) - 1):
                token_strings = tokens[end_idxs[i] : end_idxs[i + 1]]
                weights = non_zero_values[end_idxs[i] : end_idxs[i + 1]]
                output.append(dict(zip(token_strings, weights)))
            return output

    def __init__(self):
        super().__init__()
        self.special_token_ids = None
        self.tokenizer = None
        self.all_tokens = None
        self.initialized = False

    def initialize(self, context):
        self.manifest = context.manifest
        properties = context.system_properties

        # Print initialization parameters
        print(f"Initializing SparseEncodingModelHandler with model_id: {model_id}")
        print(
            f"Configuration parameters - use_l0: {use_l0}, max_bs: {max_bs}, prune_ratio: {prune_ratio}"
        )

        # load model and tokenizer
        self.device = torch.device(
            "cuda:" + str(properties.get("gpu_id"))
            if torch.cuda.is_available()
            else "cpu"
        )
        print(f"Using device: {self.device}")
        self.model = SparseEncodingModelHandler.SparseModel.from_pretrained(model_id)
        self.model.to(self.device)
        self.tokenizer = transformers.AutoTokenizer.from_pretrained(model_id)

        self.post_processor = SparseEncodingModelHandler.SparsePostProcessor(
            tokenizer=self.tokenizer
        )
        self.special_token_ids = [
            self.tokenizer.vocab[token]
            for token in self.tokenizer.special_tokens_map.values()
        ]
        self.model.set_special_token_ids(self.special_token_ids)

        self.initialized = True

    def preprocess(self, requests):
        inputSentence = []

        batch_idx = []
        for request in requests:

            request_body = request.get("body")
            if isinstance(request_body, bytearray):
                request_body = request_body.decode("utf-8")
                request_body = json.loads((request_body))

            if isinstance(request_body, list):
                inputSentence += request_body
                batch_idx.append(len(request_body))
            else:
                inputSentence.append(request_body)
                batch_idx.append(1)

        input_data = self.tokenizer(
            inputSentence,
            padding=True,
            truncation=True,
            max_length=self.tokenizer.model_max_length,
            return_tensors="pt",
            return_attention_mask=True,
            return_token_type_ids=False,
        )

        tokens = input_data["attention_mask"].sum(dim=-1).numpy().tolist()
        input_data = input_data.to(self.device)
        return {"input": input_data, "batch_l": batch_idx, "tokens": tokens}

    def inference(self, data, *args, **kwargs):
        batch_idx = data["batch_l"]
        tokens = data["tokens"]
        data_input = data["input"]

        total_samples = len(tokens)
        outputs = []

        for start_idx in range(0, total_samples, max_bs):
            end_idx = min(start_idx + max_bs, total_samples)

            batch_data = {
                "input_ids": data_input["input_ids"][start_idx:end_idx],
                "attention_mask": data_input["attention_mask"][start_idx:end_idx],
            }

            with torch.cuda.amp.autocast(), torch.no_grad():
                output = self.model(**batch_data)
                outputs.append(output)

        output = torch.cat(outputs, dim=0)
        return {"pred": output, "batch_l": batch_idx, "tokens": tokens}

    def postprocess(self, prediction):
        batch_idx = prediction["batch_l"]
        output = prediction["pred"]
        tokens = prediction["tokens"]
        output = self.post_processor(output)
        usage = [
            {"inputTokens": tokens[i], "outputTokens": 0} for i in range(len(output))
        ]

        # return the inference results to each request according to batch size
        outputs = []
        index = 0
        for b in batch_idx:
            outputs.append(
                {
                    "response": output[index : index + b],
                    "tokens": usage[index : index + b],
                }
            )
            index += b
        return outputs

    def handle(self, data, context):
        model_input = self.preprocess(data)
        model_output = self.inference(model_input)
        model_output = self.postprocess(model_output)
        return model_output

Wrap the handler folder to a tarball. And upload it to your S3 bucket.

In handler/neural_sparse_handler.py, we define the model loading, pre-process, inference and post-process. We use mixed-precision to accelerate the inference.

In handler/neural_sparse_config.yaml, we define some configs for the torch serve (include dynamic micro-batching)

In [ ]:
import os

bucket_name = "your_bucket_name"
os.system("tar -czvf neural-sparse-handler.tar.gz -C handler/ .")
os.system(
    f"aws s3 cp neural-sparse-handler.tar.gz s3://{bucket_name}/neural-sparse-handler.tar.gz"
)

## Step 3
Use SageMaker python SDK to deploy the tarball on a real-time inference endpoint

Here we use ml.g5.xlarge. It's a GPU instance with good price-performance.

Please modify the region base according to your settings

In [ ]:
# constants that can be customized for models
model_id = "opensearch-project/opensearch-neural-sparse-encoding-doc-v2-distill"
max_batch_size = "32"
prune_ratio = "0.1"

# constants related to deployment
model_name = "ns-handler"
endpoint_name = "ns-handler"
instance_type = "ml.g5.xlarge"
initial_instance_count = 1

# run this cell
import boto3
import sagemaker
from sagemaker.model import Model
from sagemaker.predictor import Predictor
from sagemaker.serializers import JSONSerializer
from sagemaker.deserializers import JSONDeserializer

role = sagemaker.get_execution_role()
sess = boto3.Session()
region = sess.region_name
smsess = sagemaker.Session(boto_session=sess)

envs = {
    "TS_ASYNC_LOGGING": "true",
    "MODEL_ID": model_id,
    "MAX_BS": max_batch_size,
    "PRUNE_RATIO": prune_ratio,
}

baseimage = sagemaker.image_uris.retrieve(
    framework="pytorch",
    region=region,
    py_version="py312",
    image_scope="inference",
    version="2.6",
    instance_type=instance_type,
)

model = Model(
    model_data=f"s3://{bucket_name}/neural-sparse-handler.tar.gz",
    image_uri=baseimage,
    role=role,
    predictor_cls=Predictor,
    name=model_name,
    sagemaker_session=smsess,
    env=envs,
)

endpoint_name = endpoint_name
predictor = model.deploy(
    instance_type=instance_type,
    initial_instance_count=initial_instance_count,
    endpoint_name=endpoint_name,
    serializer=JSONSerializer(),
    deserializer=JSONDeserializer(),
    ModelDataDownloadTimeoutInSeconds=3600,
    ContainerStartupHealthCheckTimeoutInSeconds=3600,
    VolumeSizeInGB=64,
)

print(predictor.endpoint_name)

## Step 4

After we create the endpoint, use some sample request to see how it works

In [ ]:
# run this cell
import json

body = ["Currently New York is rainy."]
amz = boto3.client("sagemaker-runtime")

response = amz.invoke_endpoint(
    EndpointName=predictor.endpoint_name,
    Body=json.dumps(body),
    ContentType="application/json",
)

res = response["Body"].read()
results = json.loads(res.decode("utf8"))
results

response:
```json
{'response': [{'has': 0.19832642376422882,
   'new': 0.9849710464477539,
   'like': 0.20112557709217072,
   'now': 0.7473171949386597,
   'state': 0.20818853378295898,
   'still': 0.26296505331993103,
   'going': 0.17759032547473907,
   'york': 1.5465646982192993,
   'water': 0.5180262327194214,
   'present': 0.24726435542106628,
   'today': 0.5316043496131897,
   'currently': 0.6706798672676086,
   'current': 0.9104140996932983,
   'dry': 0.2999960780143738,
   'rain': 1.3858059644699097,
   'weather': 1.4669378995895386,
   'climate': 0.392688512802124,
   'wet': 1.070887804031372,
   'happening': 0.3875649571418762,
   'ny': 1.4108916521072388,
   'brooklyn': 0.2983669638633728,
   'yorkshire': 0.15651951730251312,
   'manhattan': 0.969535231590271,
   'flood': 0.2403770089149475,
   'flooding': 0.4161500036716461,
   'rainfall': 0.9889746904373169,
   'damp': 0.38938602805137634,
   'moist': 0.32199856638908386,
   'mist': 0.2026219218969345,
   'precipitation': 0.5729197263717651,
   'drought': 0.41227778792381287,
   'rains': 0.8187123537063599,
   'rainy': 1.4709837436676025,
   'nyc': 1.308121681213379,
   'yorker': 0.6350979804992676,
   'monsoon': 0.6218147873878479,
   'raining': 0.9827804565429688,
   'cloudy': 0.6314691305160522,
   'nyu': 0.7196483612060547}],
 'tokens': [{'inputTokens': 8, 'outputTokens': 0}]}
```

## Step 5
> **_NOTE:_**  **This step is supposed to be executed at an instance have access to OpenSearch cluster!**

Register this SageMaker endpoint at your OpenSearch cluster

Please check the OpenSearch doc for more information. Here we provide one demo request body using access_key and secret_key. Please choose the authentication according to your use case.

### create connector

(Fill the region and predictor.endpoint_name in request body)
```json
POST /_plugins/_ml/connectors/_create
{
  "name": "test",
  "description": "Test connector for Sagemaker model",
  "version": 1,
  "protocol": "aws_sigv4",
  "credential": {
    "access_key": "your access key",
    "secret_key": "your secret key"
  },
  "parameters": {
    "region": "{region}",
    "service_name": "sagemaker",
    "input_docs_processed_step_size": 2,
  },
  "actions": [
    {
      "action_type": "predict",
      "method": "POST",
      "headers": {
        "content-type": "application/json"
      },
      "url": "https://runtime.sagemaker.{region}.amazonaws.com/endpoints/{predictor.endpoint_name}/invocations",
      "request_body": "${parameters.input}"
    }
  ],
  "client_config":{
      "max_retry_times": -1,
      "max_connection": 60,
      "retry_backoff_millis": 10
  }
}
```

### register model
```json
POST /_plugins/_ml/models/_register?deploy=true
{
  "name": "test",
  "function_name": "remote",
  "version": "1.0.0",
  "connector_id": "{connector id}",
  "description": "Test connector for Sagemaker model"
}
```